# LLM agent Hackathon 
## Alireza Ghafarollahi, MIT

# AG2: Building LLM-powered multi-agent systems

In [ ]:
!pip install ag2
!pip install openai

In [7]:
import json
import os

with open('config.json') as f:
    config = json.load(f)

OPENAI_API_KEY = config["api_key"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Example: Group chat to assist the user with a tech-related query

## Agents

- ## coordinator agent

- ## technical support agent

- ## general support agent

## Enhance our system with tools

- ## classify_query

In [8]:
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import AutoPattern

# Create Tool

In [14]:
from typing import Annotated
from autogen.agentchat.group import ReplyResult, AgentNameTarget

def classify_query(
    query: Annotated[str, "The user query to classify"],
) -> ReplyResult:
    """
    Classify a user query as technical or general.

    Technical queries involve hardware problems, software errors, system crashes, etc.
    General queries involve information requests, conceptual questions, etc.
    """
    # Simple keyword-based classification
    technical_keywords = ["error", "bug", "broken", "crash", "not working", "shutting down"]

    # Check if any technical keywords are in the query
    if any(keyword in query.lower() for keyword in technical_keywords):
        return ReplyResult(
            message="This appears to be a technical issue.",
            target=AgentNameTarget("tech_agent")  # Route to technical support
        )
    else:
        return ReplyResult(
            message="This appears to be a general question.",
            target=AgentNameTarget("general_agent")  # Route to general support
        )

# Create specialized agents and assign tool(s)

In [15]:
llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")

with llm_config:
    coordinator_agent = ConversableAgent(
        name="coordinator_agent",
        system_message="""You are a coordinator agent. For each user query,
        identify whether it is a technical issue or a general question. Route
        technical issues to the tech agent and general questions to the general agent.
        Do not provide suggestions or answers, only route the query.""",
        functions=[classify_query]
    )

    tech_agent = ConversableAgent(
        name="tech_agent",
        system_message="""You solve technical problems like software bugs
        and hardware issues."""
    )

    general_agent = ConversableAgent(
        name="general_agent",
        system_message="You handle general, non-technical support questions."
    )



# Create a User Agent 

In [16]:
user = ConversableAgent(name="user", human_input_mode="ALWAYS")

# Define the Conversation Pattern

In [17]:
pattern = AutoPattern(
    initial_agent=coordinator_agent,  # Agent that starts the conversation
    agents=[coordinator_agent, tech_agent, general_agent],
    user_agent=user,
    group_manager_args={"llm_config": llm_config}
)

# Start the group chat

In [18]:
initial_prompt = """My laptop keeps shutting down randomly. Can you help?"""

result, context_variables, last_agent = initiate_group_chat(
    pattern=pattern,
    messages=initial_prompt,
    max_rounds=10
)

user (to chat_manager):

My laptop keeps shutting down randomly. Can you help?

--------------------------------------------------------------------------------

Next speaker: coordinator_agent


>>>>>>>> USING AUTO REPLY...
coordinator_agent (to chat_manager):

***** Suggested tool call (call_KjKGHnI4Koei9qPlMSU7jPAZ): classify_query *****
Arguments: 
{"query":"My laptop keeps shutting down randomly. Can you help?"}
*******************************************************************************

--------------------------------------------------------------------------------

Next speaker: _Group_Tool_Executor


>>>>>>>> EXECUTING FUNCTION classify_query...
Call ID: call_KjKGHnI4Koei9qPlMSU7jPAZ
Input arguments: {'query': 'My laptop keeps shutting down randomly. Can you help?'}
_Group_Tool_Executor (to chat_manager):

***** Response from calling tool (call_KjKGHnI4Koei9qPlMSU7jPAZ) *****
This appears to be a technical issue.
************************************************************

Replying as user. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  EXIT


user (to chat_manager):

EXIT

--------------------------------------------------------------------------------

Next speaker: general_agent


>>>>>>>> USING AUTO REPLY...
general_agent (to chat_manager):

If you need to leave, that's perfectly fine! Feel free to return if you have more questions in the future. Take care!

--------------------------------------------------------------------------------

Next speaker: user



Replying as user. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit



>>>>>>>> TERMINATING RUN (bb49c7e8-dfb6-4201-b83a-22e2320f7b19): User requested to end the conversation

>>>>>>>> TERMINATING RUN (178ab488-54bc-401d-9fde-6bb9f5647b74): No reply generated
